# Systematic Errors and Gaussian Subsidiary measurements

Build a model Poisson$(N_\mathrm{SR} | \mu * S + (B_\mathrm{nom}+\alpha_B*B_\mathrm{systerr}) ) * \mathrm{Gaussian} ( 0 | \alpha_B, 1)$ representing a signal region SR that includes an uncertain background component $B$ that is known with a systematic uncertainty $B = B_\mathrm{nom} \pm B_\mathrm{systerr}$

The constant $B$ from previous exercises is replaced by a response function $B(\alpha_B) = B_\mathrm{nom} + \alpha_B * B_\mathrm{systerr}$ where $B_\mathrm{nom}$ is known nominal value of $B$ and $B_\mathrm{systerr}$ is the associated systematic uncertainty. 
    
The response function $B(\alpha_B)$ is defined such that if nuisance parameter $\alpha_B$ is measured with unit Gaussian uncertainty the uncertainty on $B$ is modelled as Gaussian centered at $B_\mathrm{nom}$ with a error of $B_\mathrm{systerr}$.

A subdiary Gaussian measurement is included that measured $\alpha_B$ according to a unit Gaussian distribuion. Since the observed value of the systematic uncertainty is by construction zero, that observable is included as a Global Observable in the model.

In [1]:
import ROOT
%jsroot on

Welcome to JupyROOT 6.12/06


## Build Model

Construct an empy workspace that will contain the model

In [2]:
w = ROOT.RooWorkspace("w")


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



### Construct a Poisson signal region model

Construct the response function for the background $B = B_\mathrm{nom} + \alpha_B * B_\mathrm{systerr}$

In [3]:
w.factory("expr::B('B_nom+alpha_B*B_systerr',B_nom[20],alpha_B[-5,5],B_systerr[4])") 

<ROOT.RooFormulaVar object ("B") at 0x3f8a4d0>

Construct expression for expected event count `Nexp_SR` = $\mu * S + B$, with $S$ fixed and $B$ previously defined

In [4]:
w.factory("expr::Nexp_SR('mu*S+B',mu[1,-1,10],S[10],B)")

<ROOT.RooFormulaVar object ("Nexp_SR") at 0x82d7de0>

Construct Poisson probability model for Nobs_SR given Nexp_SR

In [5]:
w.factory("Poisson::model_SR(Nobs_SR[0,100],Nexp_SR)")

<ROOT.RooPoisson object ("model_SR") at 0x83bae00>

### Construct a Gaussian subsidiary measurement

Construct expression for expected event count Nexp_CR $= \tau * B$, with $\tau$ fixed. The factor $\tau$ expresses the ratio of the CR size w.r.t. the SR size.

In [6]:
w.factory("Gaussian::model_alphaB(alphaB_nom[0],alpha_B,1)")

<ROOT.RooGaussian object ("model_alphaB") at 0x83cea20>

### Construct a joint model for the SR and CR

In [7]:
w.factory("PROD::model(model_SR,model_alphaB)")

<ROOT.RooProdPdf object ("model") at 0x83f7b60>

### Construct a dataset containing a single observed event count

Construct an empty dataset for observable Nobs

In [8]:
d = ROOT.RooDataSet("d","d",ROOT.RooArgSet(w.var("Nobs_SR")))

Set Nobs to desired observed event count

In [9]:
w.var("Nobs_SR").setVal(25)

Add observation of 25 events to dataset

In [10]:
d.add(ROOT.RooArgSet(w.var("Nobs_SR")))
getattr(w,'import')(d,ROOT.RooFit.Rename("observed_data"))

False

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing dataset d
[#1] INFO:ObjectHandling -- RooWorkSpace::import(w) changing name of dataset from  d to observed_data


## Construct a unique statistical interpretatation of the model to be used by RooStats

Create an empty ModelConfig

In [11]:
mc = ROOT.RooStats.ModelConfig("ModelConfig",w)

Define the pdf, the parameter of interest and the observables

In [12]:
mc.SetPdf(w.pdf("model"))
mc.SetParametersOfInterest(ROOT.RooArgSet(w.var("mu")))
mc.SetNuisanceParameters(ROOT.RooArgSet(w.var("alpha_B")))
mc.SetObservables(ROOT.RooArgSet(w.var("Nobs_SR")))
mc.SetGlobalObservables(ROOT.RooArgSet(w.var("alphaB_nom")))

Define the current value $\mu = 1$ as an hypothesis

In [13]:
mc.SetSnapshot(ROOT.RooArgSet(w.var("mu")))

mc.Print()


=== Using the following for ModelConfig ===
Observables:             RooArgSet:: = (Nobs_SR)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (alpha_B)
Global Observables:      RooArgSet:: = (alphaB_nom)
PDF:                     RooProdPdf::model[ model_SR * model_alphaB ] = 0.0511153
Snapshot:                
  1) 0x8861e30 RooRealVar:: mu = 1  L(-1 - 10)  "mu"



import model in the workspace

In [14]:
getattr(w,'import')(mc)

w.writeToFile("../workspaces/model.root")

False

## Fit model to the data 

*NB: Global Observables that occur in the model must be explicitly identified when using the pdf*

In [15]:
w.pdf("model").fitTo(d,ROOT.RooFit.GlobalObservables(ROOT.RooArgSet(w.var("alphaB_nom"))))

<ROOT.RooFitResult object at 0x(nil)>

[#1] INFO:Minization -- createNLL: caching constraint set under name CONSTR_OF_PDF_model_FOR_OBS_Nobs_SR with 1 entries
[#1] INFO:Minization --  Including the following contraint terms in minimization: (model_alphaB)
[#1] INFO:Minization -- The following global observables have been defined: (alphaB_nom)
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_d_with_constr) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (model_SR)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 alpha_B      0.00000e+00  1.00000e+00   -5.00000e+00  5.00000e+00
     2 mu           1.00000e+00  1.00000e+00   -1.00000e+00  1.00000e+01
 **********
 **    3 **SET ERR         0.5
 **